# Referenzbeispiel 5B

# Dateioperationen

Der Stoff umfasst Kapitel 1 bis 17 im Buch “Programmieren in C”.

## Entfernen von Kommentaren in C

In diesem Referenzbeispiel wollen wir eine Funktion eine C Präprozessors nachbilden: das Entfernen von Kommentaren aus dem Quelltext. Es gibt zwei Arten von Kommentaren:

* Mehrzeilige Kommentare: /* Ich bin ein Kommentar */
* Einzeilige Kommentrare: // Ich bin ein Kommentar

Mehrzeilige Kommentare enden immer mit einem `*/`, einzeilige mit dem Ende der Zeile, also einem `\n`.

## Programmablauf

Um diese Kommentare zu identifiezieren, wird jedes Zeichen einer Datei durchlaufen und wenn ein `/` gelesen wird, ein Status auf `COMMENT_START` gesetzt. Wenn das nächste Zeichen ein `*` oder ein `/` ist, wird der Status auf `COMMENT` gesetzt. Wenn ein `*` gefunden wird, wechselt der Status auf `COMMENT_END`. Wenn das nächste Zeichen ein `/` ist, oder ein einfaches `\n` gefunden wird, wird der Status auf `SOURCE` (Quelltext) geändert. Alle Zeichen, bei denen der Status nicht `SOURCE` ist, werden ignoriert und nicht in die zweite Datei geschrieben.

## Programm

In [ ]:
#include "stdio.h" /* für Dateioperationen */
#include "stdlib.h" /* für Finden des HOME folders */
#include "string.h" /* für string operationen */

#define MAX_PATH_LENGTH 256

/* Aufzählung für Status des derzeitigen Zeichens */
typedef enum State_e {
    SOURCE = 0,
    COMMENT_START = 1,
    COMMENT = 2,
    LINE_COMMENT = 3,
    COMMENT_END = 4
} State_t;

long removeComments(FILE*, FILE*);

int main() {
    /* Wir entfernen Kommentare aus dieser Datei */
    char sourceName[] = __FILE__; /* siehe Vorlesung 3.3 */
    
    /* Geschrieben wird in Ihr HOME Verzeichnis (Startseite am Server).
        Am Server muss nach HOME geschrieben werden, weil Sie in diesem
        Ordner keine Schreibberechtigung haben. */
    char targetName[MAX_PATH_LENGTH];
    strcat(strcpy(targetName, getenv("HOME")), "/no_comments.c");
    
    {
        FILE *source;
        FILE *target;
    
        if((source = fopen(sourceName, "r"))) {
            if((target = fopen(targetName, "w"))) {
                /* Falls etwas schief geht, Fehler ausgeben */
                if(removeComments(source, target)) {
                    fprintf(stderr, "ERROR: Could not remove comments!\n");
                    return -1;
                }
                fclose(source);
                fclose(target);
                printf("Without comments written to %s\n", targetName);
            } else {
                fprintf(stderr, "ERROR: Could not open %s\n", targetName);
                return -1;
            }
        } else {
            fprintf(stderr, "ERROR: Could not open %s\n", sourceName);
            return -1;
        }
    }
    
    return 0;
}

long removeComments(FILE* source, FILE* target) {
    long readValue = 0; /* wir brauchen long für den Wert -1 */
    State_t state = SOURCE; /* derzeitiger Status */
    State_t lastState = SOURCE; /* vorheriger Status */
    
    while((readValue = fgetc(source)) != EOF) {
        char currentChar = (char)readValue; /* char aus long machen */
        
        switch(state) {
            case SOURCE: 
                if(currentChar == '/') {
                    state = COMMENT_START;
                }
                break;
            case COMMENT_START:
                if(currentChar == '*') {
                    state = COMMENT;
                } else if(currentChar == '/') {
                    state = LINE_COMMENT; // zwei / sind ein line comment
                } else {
                    state = SOURCE; /* Wenn ein anderes Zeichen kommt, ist es wieder Quelltext */
                }
                break;
            case COMMENT:
                if(currentChar == '*') {
                    state = COMMENT_END;
                }
                break;
            case COMMENT_END:
                if(currentChar == '/') {
                    state = SOURCE;
                } else if(currentChar != '*') { /* Wenn noch ein Stern, als COMMENT_END bleiben */
                    state = COMMENT;
                }
                break;
            case LINE_COMMENT:
                if(currentChar == '\n') {
                    state = SOURCE;
                }
        }
        
        // Entscheiden ob das Zeichen ein Kommentar ist oder nicht
        if((state == SOURCE && lastState == COMMENT_START) ||
            (state == COMMENT_START && lastState == COMMENT_START)) {
            putc('/', target); /* ein '/' wurde gelesen, ist aber kein Kommentar */
        }
        if(state == SOURCE && lastState != COMMENT_END) {
            putc(currentChar, target); /* Für alle Nichtkommentare einfach kopieren */
        }
        
        lastState = state;
    }
    return 0;
}